In [2]:
import numpy as np
import pandas as pd
from __future__ import division


In [3]:
!pip install numba

In [4]:
from numba import cuda, float32
import math
import time

In [5]:
A = np.random.rand(100,1000)
B = np.random.rand(1000,100)
C= np.zeros([100,100])

In [6]:
def norm_CPU_multiplication(A,B,C):
  for i in range(len(A)):
    for j in range(len(B[0])):
      for k in range(len(B)):
        C[i][j] += A[i][k]*B[k][j]
  

In [7]:
t1=time.time()
norm_CPU_multiplication(A,B,C)
t2=time.time()
print(f"Time taken using CPU: {t2-t1} s")

Time taken using CPU: 14.474081993103027 s


In [8]:
cuda.detect()
TPB = 16

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


In [9]:
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    x, y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    if x >= C.shape[0] and y >= C.shape[1]:
        return
    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]
        # Wait until all threads finish computing
        cuda.syncthreads()
    C[x, y] = tmp

In [10]:
# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)

# Allocate memory on the device for the result
C_global_mem = cuda.to_device(C)

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel
t1 = time.time() 
%time fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
t2 = time.time()
print(f"Time taken using Cuda enabled GPU: {t2-t1} s")
cuda.synchronize()
# Copy the result back to the host
C = C_global_mem.copy_to_host()
print(C)

CPU times: user 486 ms, sys: 139 ms, total: 625 ms
Wall time: 839 ms
Time taken using Cuda enabled GPU: 0.840301513671875 s
[[250.87959658 243.27234536 240.74487816 ... 242.0475735  240.10817654
  246.740573  ]
 [251.52568975 242.37081779 247.75175142 ... 240.31095855 240.07474709
  239.41254866]
 [251.75000071 240.23767165 239.79359701 ... 233.8446009  233.4805479
  241.76428818]
 ...
 [251.99175769 241.71379013 243.3729617  ... 239.13409464 242.71306458
  243.17882511]
 [271.63980474 249.18459253 251.42040772 ... 253.80387899 255.23206535
  257.97255593]
 [261.84900164 245.73811686 244.38043513 ... 247.21284295 250.31213379
  252.36456222]]


We can see that using Cuda enabled GPU code, the time is very less as comapred to the cpu time.

In [11]:
def L2_norm(A):
  s=0
  for i in A:
    s+= i*i
  return np.sqrt(s)

In [12]:
A = np.random.random(25000000)
t1 = time.time()
%time L2_norm(A)
t2 = time.time()
print('Time taken in CPU Norm calculation of vector: {}  s'.format((t2-t1)))

CPU times: user 8.02 s, sys: 33.3 ms, total: 8.06 s
Wall time: 8.08 s
Time taken in CPU Norm calculation of vector: 8.079189777374268  s


In [13]:
@cuda.jit
def cuda_norm(a, out):
    temp_sum = 0
    x = cuda.grid(1)
    if(x<a.size):
        temp_sum += a[x]**2
    out[0] = temp_sum**0.5


In [14]:
out = np.zeros(1)
threadsperblock = 256
blockspergrid = math.ceil(A.shape[0]/threadsperblock)
t1 = time.time()
cuda_norm[blockspergrid, threadsperblock](A, out)
t2 = time.time()
print(out[0])
print(f"Time to execute in GPU: {t2-t1} s")

0.7260255853022367
Time to execute in GPU: 0.3567492961883545 s


In [15]:
A = np.random.rand(500,1000)
x = np.ones([1000,1])
temp = np.zeros([1000,1])
temp1 = np.zeros([1000,1])

In [17]:
A_global_mem = cuda.to_device(A)
x_global_mem = cuda.to_device(x)
temp_global_mem = cuda.to_device(temp)
temp1_global_mem = cuda.to_device(temp1)

In [18]:
@cuda.jit
def normalize(x, fac, normalized_x):
    fac[0][0] = 0
    fac[0][0] += x[0][0]
    for num in x:
        tempmax = abs(num[0])
        fac[0][0] = max(fac[0][0], tempmax)
    x_max = x[0][0]
    for num in x:
        x_max = max(x_max, num[0])
    for i in range(len(x)):
        normalized_x[i][0] = x[i][0]/x_max

In [20]:
threadsperblock = 256
blockspergrid = math.ceil(A.shape[0] / threadsperblock)

t1=time.time()
for i in range(8):
    fast_matmul[(2,), (4, 4)](A_global_mem, x_global_mem, temp1_global_mem); cuda.synchronize()
    lambda_1=np.zeros([1,1])
    d_lambda_1=cuda.to_device(lambda_1)
    normalize[blockspergrid, threadsperblock](temp1_global_mem,d_lambda_1,temp_global_mem)
    print('Eigenvalue {}:{}'.format(i+1, d_lambda_1.copy_to_host()[0][0]))
    x_global_mem = temp_global_mem
t2=time.time()
print('Time taken in cuda enabled GPU Power Method calculation of Eigen vector: {} s'.format(t2-t1))

Eigenvalue 1:376.1997198606841
Eigenvalue 2:247.82841815426946
Eigenvalue 3:247.78349506482482
Eigenvalue 4:247.78333277255297
Eigenvalue 5:247.78333136439323
Eigenvalue 6:247.78333136439323
Eigenvalue 7:247.78333136439323
Eigenvalue 8:247.78333136439323
Time taken in cuda enabled GPU Power Method calculation of Eigen vector: 0.35503721237182617 s
